In [ ]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine

## Extract CSVs into DataFrames

In [ ]:
salaries_file = 'Resources/degrees-that-pay-back.csv'
grads_file = 'Resources/recent-grads.csv'

In [ ]:
salaries_df = pd.read_csv(salaries_file)
salaries_df['id'] = salaries_df.index
salaries_df.head()

In [ ]:
grads_df = pd.read_csv(grads_file,index_col=False)
grads_df['id'] = grads_df.index
grads_df.head()

## Transform DataFrames

In [ ]:
salaries_df_cols =["id","Undergraduate Major", "Starting Median Salary", "Mid-Career Median Salary", "Mid-Career 10th Percentile Salary", "Mid-Career 90th Percentile Salary" ]
salaries_transformed = salaries_df[salaries_df_cols].copy()

salaries_transformed_rename = salaries_transformed.rename(columns={"id":"id",
                                                                   "Undergraduate Major": "major",
                                                                    "Starting Median Salary": "starting_median",
                                                                     "Mid-Career Median Salary": "mid_sal_median",
                                                                     "Mid-Career 10th Percentile Salary": "mid_sal_lower",
                                                                      "Mid-Career 90th Percentile Salary": "mid_sal_upper" })

salaries_transformed_rename["major"] = salaries_transformed_rename["major"].str.upper()
salaries_transformed_rename.reset_index(drop=True)
salaries_transformed_rename.set_index("id",inplace=True)
salaries_transformed_rename.head()

In [ ]:
grads_df_cols=["id","Rank","Major", "Total", "Employed"]
grads_transformed = grads_df[grads_df_cols].copy()

grads_transformed_rename = grads_transformed.rename(columns={"id":"id",
                                                             "Rank": "rank",
                                                         "Major": "major",
                                                         "Total": "total_grads",
                                                         "Employed": "employed",
                                                            })

grads_transformed_rename.reset_index(drop=True)
grads_transformed_rename.set_index("id",inplace=True)
grads_transformed_rename.head()

In [ ]:
#Create Database Connection:
connection_string = "postgres:postgres@localhost:5432/graduate_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

In [ ]:
grads_transformed_rename.to_sql(name='grads', con=engine, if_exists='append', index=True) 

In [ ]:
salaries_transformed_rename.to_sql(name='salary', con=engine, if_exists='append', index=True) 

## Import combined query table and transform currency values to numeric

In [ ]:
combined = 'Resources/combined.csv'
combined_df = pd.read_csv(combined)
combined_df["starting_median"] = (combined_df['starting_median'].replace( '[\$,)]','', regex=True ).replace( '[(]','-', regex=True ).astype(float))
combined_df["mid_sal_median"] = (combined_df['mid_sal_median'].replace( '[\$,)]','', regex=True ).replace( '[(]','-', regex=True ).astype(float))
combined_df["mid_sal_lower"] = (combined_df['mid_sal_lower'].replace( '[\$,)]','', regex=True ).replace( '[(]','-', regex=True ).astype(float))
combined_df["mid_sal_upper"] = (combined_df['mid_sal_upper'].replace( '[\$,)]','', regex=True ).replace( '[(]','-', regex=True ).astype(float))

combined_df

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

### Expected Starting Salaries

In [ ]:
fig, ax = plt.subplots()
multi = ['#2a5495', '#07a64c', '#e979ad', '#d88432', '#2a5495',
               '#b7040e', '#82c5db', '#cd065d', '#b9c09b', '#4b117f']
major_values = combined_df["major"]
salary_values = combined_df["starting_median"].sort_values(ascending=False)
ax.bar(major_values, salary_values, align="center", color = multi)
ax.set_title("Median Starting Salary by Major", fontsize=14)
ax.set_xlabel("Major", fontsize=14)
ax.set_ylabel("Starting Median Salary ($)", fontsize=14)
ax.set_xticklabels(major_values, rotation=90, weight='bold', va="center", ha="center")
ax.tick_params(axis="x", pad=-159)
ax.grid()
plt.subplots_adjust(bottom=-0.5, right=1.2)
plt.savefig("Resources/salary_bar_chart.png", bbox_inches='tight')
plt.show()

### Expected Salary Trajectories

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

#Plot salary trajectories for selected majors
# selected_majors = ["CHEMICAL ENGINEERING", "CIVIL ENGINEERING", "PHYSICS", "JOURNALISM", "SOCIOLOGY", "MUSIC"]
salary_types = ["Starting_Median", "Mid_Lower", "Mid-Career_Median", "Mid_Upper" ]

#-----------------------
CHEM_ENG_values = combined_df[["starting_median", "mid_sal_lower", "mid_sal_median", "mid_sal_upper"]].loc[combined_df['major']=="CHEMICAL ENGINEERING"]
CIV_ENG_values = combined_df[["starting_median", "mid_sal_lower", "mid_sal_median", "mid_sal_upper"]].loc[combined_df['major']=="CIVIL ENGINEERING"] 
PHYSICS_values = combined_df[["starting_median", "mid_sal_lower", "mid_sal_median", "mid_sal_upper"]].loc[combined_df['major']=="PHYSICS"] 
JOURNALISM_values = combined_df[["starting_median", "mid_sal_lower", "mid_sal_median", "mid_sal_upper"]].loc[combined_df['major']=="JOURNALISM"] 
SOCIOLOGY_values = combined_df[["starting_median", "mid_sal_lower", "mid_sal_median", "mid_sal_upper"]].loc[combined_df['major']=="SOCIOLOGY"] 
MUSIC_values = combined_df[["starting_median", "mid_sal_lower", "mid_sal_median", "mid_sal_upper"]].loc[combined_df['major']=="MUSIC"] 
#-----------------------

CHEM_ENG_handle, = plt.plot(salary_types, CHEM_ENG_values.iloc[0], marker="^", color="#2a5495", label="CHEM_ENG")
CIV_ENG_handle = plt.plot(salary_types, CIV_ENG_values.iloc[0], marker=".", color="#82c5db", label="CIV_ENG")
PHYSICS_handle = plt.plot(salary_types, PHYSICS_values.iloc[0], marker = "+", color="#07a64c", label="PHYSICS")
JOURNALISM_handle = plt.plot(salary_types, JOURNALISM_values.iloc[0], marker= "*", color="#cd065d", label="JOURNALISM")
SOCIOLOGY_handle = plt.plot(salary_types, SOCIOLOGY_values.iloc[0], marker = "^", color ="yellowgreen" , label="SOCIOLOGY")
MUSIC_handle = plt.plot(salary_types, MUSIC_values.iloc[0], marker ='+', color="orange", label = "MUSIC")

plt.xlabel("Salary Type", fontsize=14)
plt.ylabel("Salary($)", fontsize=14)
plt.grid()
plt.legend()
plt.title("Salary Trajectories for Selected Majors", fontsize=14)
plt.savefig("Resources/salary_trajectories.png")
plt.show()